In [2]:
# import pytorch libraries
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import GATConv
from torch_geometric.data import Data
from torch_geometric.deprecation import deprecated

import os
import csv
import copy
import math
import time
import random
import argparse
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tools.eval_measures import mse
from decimal import Decimal
from scipy import stats
from scipy.stats import norm
from scipy.stats import spearmanr
from scipy.spatial.distance import cosine
import matplotlib.pyplot as plt

In [3]:
# initial settings
# train the model on GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

region_pick = ['Amygdala', 'Anterior_cingulate_cortex_BA24', 'Caudate_basal_ganglia', 
               'Cerebellar_Hemisphere', 'Frontal_Cortex_BA9', 'Hippocampus', 'Hypothalamus', 
               'Nucleus_accumbens_basal_ganglia', 'Putamen_basal_ganglia', 'Substantia_nigra']

# name difference
def find_allen_name(gtex_region):
    if gtex_region=='Cerebellar_Hemisphere':
        allen_name = 'Cerebellum'
    elif gtex_region=='Frontal_Cortex_BA9':
        allen_name = 'Cortex'
    else:
        allen_name = gtex_region
        
    return allen_name

def find_gtex_name(allen_name):
    if allen_name=='Cerebellum':
        gtex_region = 'Cerebellar_Hemisphere'
    elif allen_name=='Cortex':
        gtex_region = 'Frontal_Cortex_BA9'
    else:
        gtex_region = allen_name
        
    return gtex_region

# settings
all_ids = ['10021', '12876', '14380', '15496', '15697', '9861']

# path
allen_data_path = '/project/pi_rachel_melamed_uml_edu/Jianfeng/Allen/data/allen_data/allen/'
save_path = '/project/pi_rachel_melamed_uml_edu/Jianfeng/Allen/data/allen_data/quantile_normalized_allen/'

GeneExpression_allen_dict = {}
# iterate over all 6 subjects
for i in range(len(all_ids)):
    donor = all_ids[i]
    file_name = save_path + "normalized_expr_" + donor + ".csv"
    normalized_mat = pd.read_csv(file_name, header = 0)
    normalized_mat = normalized_mat.set_index('gene_symbol')
    GeneExpression_allen_dict[donor] = normalized_mat
    
    
ontology_path = allen_data_path + 'normalized_microarray_donor' + '9861' + '/Ontology.csv'
ontology = pd.read_csv(ontology_path, header = 0)
# From the ontology file, find the sub-regions in allen under gtex region
gtex_map_path = allen_data_path + "map_gtex_structure.txt"
gTex_map_dict = {}
print("Total number of regions in allen ontology:", ontology.shape[0])
for i in open(gtex_map_path):
    i = i.strip().split("\t")
    gtex_region = i[0].strip()
    allen_region = i[1].strip()
    if((allen_region == "none?") | (allen_region == 'pituitary body')):
        continue
    covered_allen_region = ontology.loc[(ontology['name']==allen_region) | ontology['structure_id_path'].str.startswith(ontology.loc[ontology['name']==allen_region, 'structure_id_path'].values[0]), 'id']
    gTex_map_dict[gtex_region] = covered_allen_region.tolist()
    print(gtex_region, "-->", allen_region, ";  number of regions in allen:", len(covered_allen_region))
print("\n")
    
    
intersected_region = GeneExpression_allen_dict['9861'].columns.tolist()
used_intersected_region_dict = {}
# unseen_intersected_region_dict = {}
for gtex_region, covered_allen_region in gTex_map_dict.items():
    used_region_list = [x for x in intersected_region if int(x) in covered_allen_region]
    used_intersected_region_dict[gtex_region] = used_region_list
    print(gtex_region, " # regions expired:", len(used_region_list))
num_used_region = sum(len(value) for value in used_intersected_region_dict.values())
print("Total number of intersected region between allen and gtex:", len(intersected_region))
print("Total number of used allen region for generating regions for gtex:", num_used_region)
print("Total number of unseen allen regions when generating regions for gtex:", len(intersected_region)-num_used_region)
print("\n")


# read the summarized allen data (in gtex format) into a dictionary
save_path = '/project/pi_rachel_melamed_uml_edu/Jianfeng/Allen/data/allen_data/quantile_normalized_allen/'
# find the file and read it into a dictionary
summarized_gtex_dict = {}
for file_name in os.listdir(save_path):
    if file_name.endswith('-gtex.txt'):
        key = file_name.split('-gtex.txt')[0]
        file_path = os.path.join(save_path, file_name)
        mat = pd.read_csv(file_path, sep='\t', index_col=0)
        #mat = mat.iloc[:-1]
        # Store the dataframe in the dictionary with the key
        summarized_gtex_dict[key] = mat
        

# Load gtex data
data_dir = '/project/pi_rachel_melamed_uml_edu/Jianfeng/Allen/src/Pytorch/12052023/'
gt = pd.read_csv(data_dir+"new_normed_gtex_gtex_allen_gene.txt", low_memory=False, index_col=0, sep="\t")

# build a dictionary to count the freq of each subject 
sample_subject_list = gt.loc['subject'].tolist()
subject_count_dict = {}
for s in sample_subject_list:
    if s in subject_count_dict:
        subject_count_dict[s] = subject_count_dict[s] + 1
    else:
        subject_count_dict[s] = 1
# build a dictionary to count the freq of each region
sample_region_list = gt.loc['region'].tolist()
region_count_dict = {}
for s in sample_region_list:
    if s in region_count_dict:
        region_count_dict[s] = region_count_dict[s] + 1
    else:
        region_count_dict[s] = 1  

# find the subjects that have all 10 regions
pick_subject = [s for s, c in subject_count_dict.items() if c==10]
# build a dictionary for exp data for each subject in gtex who has all 10 brain regions
exp_gtex_dict = {}
for subject in pick_subject:
    submat = gt[gt.columns[gt.iloc[1]==subject]]
    submat.columns = submat.loc['region',:]
    submat = submat.iloc[2:,]
    submat.index.names = ['gene_id']
    submat = submat.sort_values(by=['gene_id'])
    submat = submat[region_pick]
    # And also, transform the dataframe in gtex from strings to numbers
    submat = submat.apply(pd.to_numeric, errors='ignore')
    # Take the average if more than 1 sample have the same gene names
    submat = submat.groupby(submat.index).mean()
    exp_gtex_dict[subject] = submat
# find 30 gtex subjects
sub_all_ids = list(exp_gtex_dict.keys())
    
    
# gene_module = pd.read_csv(allen_data_path+'41593_2015_BFnn4171_MOESM97_ESM.csv')
allen_gene_list = GeneExpression_allen_dict['9861'].index
gtex_gene_list = exp_gtex_dict['GTEX-N7MT'].index
overlapped_gene_list = [x for x in gtex_gene_list if x in allen_gene_list]  # 15044 genes here
# allen subject gene expression profile on the overlapped genes
exp_allen_dict = {}
for key, mat in GeneExpression_allen_dict.items():
    exp_allen_dict[key] = mat.loc[overlapped_gene_list]
# summarized gtex info for allen subjects on the overlapped genes
summ_gtex_info = {}
for key, mat in summarized_gtex_dict.items():
    summ_gtex_info[key] = mat.loc[overlapped_gene_list]
# rename the Cerebellum to Cerebellar_Hemisphere and Cortex to Frontal_Cortex_BA for allen people
for subject, mat in summ_gtex_info.items():
    mat.columns = exp_gtex_dict['GTEX-N7MT'].columns


# gene embeddings
g_emb_error = 0.035
g_emb_size = 2 ** 4
g_emb_path = '/project/pi_rachel_melamed_uml_edu/Jianfeng/Allen/src/Pytorch/02162024/ATG_91_103/'
g_emb_name = f'allen_gtex_gene_emb_all6subjects_size_{g_emb_size}_pearson_err_{g_emb_error}_intersected103.csv'
# np.savetxt(g_emb_path+g_emb_name, pretrain_g_emb, delimiter=',')
# read the pretrained gene embedding
pretrain_g_emb = np.genfromtxt(g_emb_path+g_emb_name, delimiter=',', dtype=np.float32)
import pickle
# Load the gene names from the file
with open(g_emb_path+g_emb_name+'_genenames.pkl', 'rb') as file:
    gene_emb_names_list = pickle.load(file)
    

# Build the edge list
# From the Ontology find the node relationship list
onto_file_path = 'normalized_microarray_donor10021/Ontology.csv'
onto_file_path = os.path.join(allen_data_path, onto_file_path)
ontology = pd.read_csv(onto_file_path)
ontology_id = ontology.loc[:, ['id', 'parent_structure_id']]
# set the parent node of 4005 to -1
ontology_id.iloc[0,1] = -1
ontology_id['parent_structure_id'] = ontology_id['parent_structure_id'].astype(int)
# View the nodes in a hierarchical way
node_child = [int(x) for x in intersected_region]
all_node = []
for i in range(1,20):
    if i==1:
        print(f"level {i}: {len(node_child)}")
        print(node_child)
        all_node.append(node_child)
    if len(node_child)==1:
        break
    if i!=1:
        node_parent = []
        for node in node_child:
            pos = ontology_id['id'].index[ontology_id['id']==node]
            # skip if it's already the ancestor
            if len(pos)==0: continue
            parent = ontology_id['parent_structure_id'][pos].values[0]
            node_parent.append(parent)
        node_parent = set(node_parent)
        node_child = [x for x in node_parent]
        print(f"level {i}: {len(node_child)}")
        print(node_child)
        all_node.append(node_child)
repeated_nodes = [x for y in all_node for x in y]
pick_nodes = set(repeated_nodes)
print(f"There are {len(pick_nodes)} nodes in total, including the dummy node for 4005\n")

pick_nodes = [x for x in pick_nodes]
pick_nodes.sort()
# exclude the ancestor node (4005) and the '-1' node
intersected_nodes_child = pick_nodes[2:]
child_nodes_chr = list(exp_allen_dict['9861'].columns)
child_nodes = [int(x) for x in child_nodes_chr]
# append other hyper-level nodes to the pick_nodes
for x in intersected_nodes_child:
    if x not in child_nodes:
        child_nodes.append(x)
# find the parent nodes for the pick_nodes
parent_nodes = []
for x in child_nodes:
    pos = ontology_id['id'].index[ontology_id['id']==x][0]
    parent = ontology_id['parent_structure_id'][pos]
    parent_nodes.append(parent)
    
print(f'Now we start to trim the graph:')
    
for _ in range(len(parent_nodes)):
    length = len(parent_nodes)
    for i in range(length):
        cid = child_nodes[i]
        pid = parent_nodes[i]
        if pid!=4005:
            # find how many children this parent node has
            count1 = parent_nodes.count(pid)
            # if this count is more than one, we don't remove this node
            if count1 > 1:
                continue
            # if this parent node only has one child, we remove it
            else:
                # find the position of this parent node in the children node list
                pidx = child_nodes.index(pid)
                # find the grandparent
                ppid = parent_nodes[pidx]
                # remove this parent and directly connect the child to its grandparent
                child_nodes[pidx] = cid
                child_nodes.pop(i)
                parent_nodes.pop(i)
                break
    if len(parent_nodes)==length:
        break
        
# put the leaves at the beginning
initial_nodes_chr = list(exp_allen_dict['9861'].columns)
new_child_nodes = [int(x) for x in initial_nodes_chr]
new_parent_nodes = []
for x in child_nodes:
    if x not in new_child_nodes:
        new_child_nodes.append(x)
for x in new_child_nodes:
    new_parent_nodes.append(parent_nodes[child_nodes.index(x)])

# put all nodes together in order so we can re-assign node id
all_nodes = new_child_nodes.copy()
for x in new_parent_nodes:
    if x not in all_nodes:
        all_nodes.append(x)
        
print(f'After cleaning: {len(all_nodes)} node left (dummy node not included)!\n')

# re-index all the nodes and all the dataframe
child_nodes_idx = []
parent_nodes_idx = []
for node in new_child_nodes:
    child_nodes_idx.append(all_nodes.index(node))
for node in new_parent_nodes:
    parent_nodes_idx.append(all_nodes.index(node))

Total number of regions in allen ontology: 1839
Amygdala --> amygdala ;  number of regions in allen: 135
Anterior_cingulate_cortex_BA24 --> cingulate gyrus, frontal part ;  number of regions in allen: 7
Caudate_basal_ganglia --> caudate nucleus ;  number of regions in allen: 10
Cerebellum --> cerebellum ;  number of regions in allen: 95
Cortex --> frontal lobe ;  number of regions in allen: 87
Hippocampus --> hippocampal formation ;  number of regions in allen: 30
Hypothalamus --> hypothalamus ;  number of regions in allen: 176
Nucleus_accumbens_basal_ganglia --> nucleus accumbens ;  number of regions in allen: 3
Putamen_basal_ganglia --> putamen ;  number of regions in allen: 3
Substantia_nigra --> substantia nigra ;  number of regions in allen: 7


Amygdala  # regions expired: 6
Anterior_cingulate_cortex_BA24  # regions expired: 2
Caudate_basal_ganglia  # regions expired: 3
Cerebellum  # regions expired: 9
Cortex  # regions expired: 14
Hippocampus  # regions expired: 6
Hypothalamus  

In [33]:
parent_nodes[94]

4277

In [32]:
child_nodes.index(4291)

94

In [5]:
file_dir = '/project/pi_rachel_melamed_uml_edu/Jianfeng/Allen/src/Pytorch/02162024/ATG_8_or_less/'
file_name = '2011-12-16203C-Supplementary_Table2.xls'
Allen_hier_tree = pd.read_excel(file_dir+file_name, sheet_name=1)
filtered_tree = Allen_hier_tree[Allen_hier_tree['ID'].isin(all_nodes)]
filtered_tree = filtered_tree.drop(filtered_tree.columns[2:6], axis=1)
filtered_tree = filtered_tree.dropna(axis=1, how='all')
filtered_tree = filtered_tree.reset_index(drop=True)

In [6]:
filtered_tree

,ID,Acronym,Structures,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,4005,Br,Brain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4006,GM,NaN,Grey Matter,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4007,Tel,NaN,NaN,Telencephalon,NaN,NaN,NaN,NaN,NaN,NaN
3,4008,Cx,NaN,NaN,NaN,Cerebral Cortex,NaN,NaN,NaN,NaN,NaN
4,4009,FL,NaN,NaN,NaN,NaN,Frontal Lobe,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
153,9587,MeRF,NaN,NaN,NaN,medullary reticular formation,NaN,NaN,NaN,NaN,NaN
154,9598,GiRt,NaN,NaN,NaN,NaN,NaN,"gigantocellular group, Left",NaN,NaN,NaN
155,9614,LMRt,NaN,NaN,NaN,NaN,NaN,"lateral medullary reticular group, Left",NaN,NaN,NaN
156,9677,Sp5,NaN,NaN,NaN,NaN,"spinal trigeminal nucleus, Left",NaN,NaN,NaN,NaN


In [21]:
file_dir = '/project/pi_rachel_melamed_uml_edu/Jianfeng/Allen/src/Pytorch/02162024/ATG_8_or_less/'
file_name = '2011-12-16203C-Supplementary_Table2.xls'
Allen_hier_tree = pd.read_excel(file_dir+file_name, sheet_name=1)
filtered_tree = Allen_hier_tree[Allen_hier_tree['ID'].isin(all_nodes)]
filtered_tree = filtered_tree.drop(filtered_tree.columns[2:6], axis=1)
filtered_tree = filtered_tree.dropna(axis=1, how='all')
filtered_tree = filtered_tree.reset_index(drop=True)

filtered_tree['Tissue with gene expression'] = np.nan
filtered_tree['Mapped GTEx tissue'] = np.nan
allen_tissue_w_exp = exp_allen_dict['9861'].columns.tolist()
for i in range(filtered_tree.shape[0]):
    allen_id = filtered_tree['ID'][i]
    # tell if this region has expression or not
    if str(allen_id) in allen_tissue_w_exp:
        filtered_tree.loc[i, 'Tissue with gene expression'] = "1"
    else:
        filtered_tree.loc[i, 'Tissue with gene expression'] = "0"
    # see the mapped GTEx tissue
    for k in list(gTex_map_dict.keys()):
        if allen_id in gTex_map_dict[k]:
            filtered_tree.loc[i, 'Mapped GTEx tissue'] = find_gtex_name(k)
            continue
            
filtered_tree.columns = ['ID', 'Acronym', 'Allen Level 1 Tissue', 'Allen Level 2 Tissue', 'Allen Level 3 Tissue', 
                         'Allen Level 4 Tissue', 'Allen Level 5 Tissue', 'Allen Level 6 Tissue', 'Allen Level 7 Tissue', 
                         'Allen Level 8 Tissue', 'Allen Level 9 Tissue', 'Tissue with gene expression', 'Mapped GTEx tissue']

# remove key word ", Left" for Allen tissues
filtered_tree = filtered_tree.applymap(
    lambda x: x.replace(", Left", "") if isinstance(x, str) else x
)

In [23]:
file_name = 'Allen_GTEx_tissue_mapping.csv'
filtered_tree.to_csv(file_dir+file_name, index=False)

In [22]:
filtered_tree

,ID,Acronym,Allen Level 1 Tissue,Allen Level 2 Tissue,Allen Level 3 Tissue,Allen Level 4 Tissue,Allen Level 5 Tissue,Allen Level 6 Tissue,Allen Level 7 Tissue,Allen Level 8 Tissue,Allen Level 9 Tissue,Tissue with gene expression,Mapped GTEx tissue
0,4005,Br,Brain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
1,4006,GM,NaN,Grey Matter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
2,4007,Tel,NaN,NaN,Telencephalon,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
3,4008,Cx,NaN,NaN,NaN,Cerebral Cortex,NaN,NaN,NaN,NaN,NaN,0,NaN
4,4009,FL,NaN,NaN,NaN,NaN,Frontal Lobe,NaN,NaN,NaN,NaN,0,Frontal_Cortex_BA9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,9587,MeRF,NaN,NaN,NaN,medullary reticular formation,NaN,NaN,NaN,NaN,NaN,0,NaN
154,9598,GiRt,NaN,NaN,NaN,NaN,NaN,gigantocellular group,NaN,NaN,NaN,1,NaN
155,9614,LMRt,NaN,NaN,NaN,NaN,NaN,lateral medullary reticular group,NaN,NaN,NaN,1,NaN
156,9677,Sp5,NaN,NaN,NaN,NaN,spinal trigeminal nucleus,NaN,NaN,NaN,NaN,1,NaN
